<a href="https://colab.research.google.com/github/dsba6010-llm-applications/los-tres-amigos/blob/main/Syllabi_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tests

## Set up

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
SYLLABI_DOCS= '/content/drive/MyDrive/Syllabi'
import sys
sys.path.append(SYLLABI_DOCS)
!pip install ordered-set
!pip install faiss-cpu
!pip install whoosh
!pip install openai
!pip install tokenizers
!pip install transformers
!pip install openai
import syllabi_store as sc
import pandas as pd

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
from os import symlink
import re
import requests
from bs4 import BeautifulSoup
import json
import logging
from openai import OpenAI
from google.colab import userdata

import requests
#import PyPDF2
from io import BytesIO
from copy import deepcopy

# Google Collab Goodies
from IPython.display import display_markdown, display_html
from google.colab import data_table
data_table.enable_dataframe_formatter()

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s -  %(module)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
formatter=logging.Formatter('%(asctime)s -  %(module)s - %(levelname)s - %(message)s')
logger = logging.getLogger("my_logger")

# Create a handler to output logs to console (like print)
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

import google.generativeai as genai
GEMINI_MODEL_VERSION='models/gemini-1.5-flash'
gemini_key = userdata.get('GeminiAPI')
genai.configure(api_key=gemini_key)
model=genai.GenerativeModel(GEMINI_MODEL_VERSION)

# Create S_Store
s_store = sc.SyllabiStore()

## Tests

### PyTests


In [3]:


def semantic_search(phrase,k=5):
    items = s_store.semantic_search(sc.CD_CONTENT,s_store.get_embedding(phrase,query_convert=True), k)
    table = list()
    for item in items:
        obj=dict()
        obj["id"] = item["item"]

        chunk = s_store.corpus.partitions['syllabi-chunked'].chunks[item["item"]]
        obj['title']=chunk.metadata["course_title"]
        obj['number']=chunk.metadata["course_number"]
        obj['instructor']=chunk.metadata["instructor"]
        obj['content']=chunk.get_v_content()
        obj["distance"]=item["distance"]
        table.append(obj)

    return table

from whoosh.qparser import QueryParser
def find_by_instructor(instructor):
    query = QueryParser("instructor", s_store.i_indexes[sc.KW_INDEX].schema).parse(instructor)
    results = s_store.keyword_search(sc.KW_INDEX,query)
    table = list()
    for result in results:
        obj=dict(result)
        chunk_id = f'{obj["id"]}.full'
        chunk = s_store.corpus.partitions['syllabi-full'].chunks[chunk_id]
        obj['content']=chunk.get_i_content()
        table.append(obj)
    return table

def find_by_course_number(course_number):
    query = QueryParser("course_number", s_store.i_indexes[sc.KW_INDEX].schema).parse(course_number)
    results = s_store.keyword_search(sc.KW_INDEX,query)
    table = list()
    for result in results:
        obj=dict(result)
        chunk_id = f'{obj["id"]}.full'
        chunk = s_store.corpus.partitions['syllabi-full'].chunks[chunk_id]
        obj['content']=chunk.get_i_content()
        table.append(obj)
    return table

def get_ragified_answer(message,rag_content):
  messages = [
    {
        "role": "system",
        "content": "Please use the list of documents in rag content to answer the question from the user",
    },
    {
        "role": "rag content",
        "content": rag_content,
    },
    {
        "role": "user",
        "content": message,
    },
  ]

  response = model.generate_content([message,rag_content])
  return response.text


# Tests that find content that will be used to Ragify the Prompt

def test_learn_python():
    """I will learn Python"""
    result = semantic_search("Which course will teach me Python")
    assert "python" in result[0]["content"].lower(), "Expected Python in result"

def test_group_project():
    """Has Group Project"""
    result = semantic_search("Which course will have a group project")
    assert "group" in result[0]["content"].lower() and "project" in result[0]["content"].lower(), "Expected group and project"

def test_taught_by_ryan():
    """Taught by Ryan Wesslen"""
    result = semantic_search("Which course is taught by Ryan Wesslen")
    assert "ryan wesslen" in result[0]["instructor"].lower(), "Expected Ryan Wesslen in result"

def test_taught_by_rick():
    """Taught by Rick Chakra"""
    result = find_by_instructor("Rick Chakra")
    assert "rick chakra" in result[0]["instructor"].lower(), "Expected Rick Chakra in Result"


def test_ryan_grading():
    """How does Ryan Wesslen Grade"""
    result = find_by_instructor("Ryan Wesslen")
    messages = "What is the grading policy for all the courses provided"
    rag_content = list()
    content=""
    for item in result:
      content += "Title: " + item["id"] + "\n" + item["content"] + "\n\n"
      rag_content.append(dict({
          "title":item["id"],
          "content":item["content"]
      }))
    response = get_ragified_answer(messages,content)
    display_markdown("### <font color='#699'>Debug output</font>",raw=True)
    display_markdown(response,raw=True)
    assert "exam" in response.lower(), "Expected Exam Result"

def test_rick_grading():
    """How does Rick Chakra Grade"""
    result = find_by_instructor("Rick Chakra")
    messages = "What is the grading policy for all the courses provided"
    rag_content = list()
    content=""
    for item in result:
      content += "Title: " + item["id"] + "\n" + item["content"] + "\n\n"
      rag_content.append(dict({
          "title":item["id"],
          "content":item["content"]
      }))
    response = get_ragified_answer(messages,content)
    display_markdown("### <font color='#699'>Debug output</font>",raw=True)
    display_markdown(response,raw=True)
    assert "exam" in response.lower(), "Expected Exam Result"

def test_lang_6201():
    """What Languages are taught in 6201"""
    result = find_by_course_number("6201")
    messages = "What programming languages are used in the courses provided"
    rag_content = list()
    content=""
    for item in result:
      content += "Title: " + item["id"] + "\n" + item["content"] + "\n\n"
      rag_content.append(dict({
          "title":item["id"],
          "content":item["content"]
      }))
    response = get_ragified_answer(messages,content)
    display_markdown("### <font color='#699'>Debug output</font>",raw=True)
    display_markdown(response,raw=True)
    assert "language" in response.lower(), "Expected Language in Result"

def test_topics_6211():
    """What Topics are taught in 6211"""
    result = find_by_course_number("6211")
    messages = "What topics are taught in the courses provided"
    rag_content = list()
    content=""
    for item in result:
      content += "Title: " + item["id"] + "\n" + item["content"] + "\n\n"
      content += item["content"] + "\n"
      rag_content.append(dict({
          "title":item["id"],
          "content":item["content"]
      }))
    response = get_ragified_answer(messages,content)
    display_markdown("### <font color='#699'>Debug output</font>",raw=True)
    display_markdown(response,raw=True)
    assert "topic" in response.lower(), "Expected Topic in Result"

def test_absence_6165():
    """What Absence Policies are in 6165"""
    result = find_by_course_number("6165")
    messages = "What are the absence policies the courses provided"
    rag_content = list()
    content=""
    for item in result:
      content += "Title: " + item["id"] + "\n" + item["content"] + "\n\n"
      content += item["content"] + "\n"
      rag_content.append(dict({
          "title":item["id"],
          "content":item["content"]
      }))
    response = get_ragified_answer(messages,content)
    display_markdown("### <font color='#699'>Debug output</font>",raw=True)
    display_markdown(response,raw=True)
    assert "absence" in response.lower(), "Expected Absence in Result"

def test_text_book_6156():
    """What Text Books are used in 6156"""
    result = find_by_course_number("6156")
    messages = "What text books are used in the syllabi provided"
    rag_content = list()
    content=""
    for item in result:
      content += "Title: " + item["id"] + "\n" + item["content"] + "\n\n"
      content += item["content"] + "\n"
      rag_content.append(dict({
          "title":item["id"],
          "content":item["content"]
      }))
    response = get_ragified_answer(messages,content)
    display_markdown("### <font color='#699'>Debug output</font>",raw=True)
    display_markdown(response,raw=True)
    assert "text" in response.lower(), "Expected Text in Result"


def run_tests(test_functions):
    test_no = 1
    for test in test_functions:
        display_markdown(f"## <font color='#669'>{test_no}</font>) {test.__name__}", raw=True)
        test_no += 1
        try:
            test()
            display_markdown(f"### {test.__name__} <font color='#699'>Passed</font>", raw=True)
        except AssertionError as e:
            display_markdown(f"### {test.__name__} <font color='#966'>Failed: {e}</font>", raw=True)

### Run Tests

In [4]:
# Automatically find all functions starting with 'test_'
test_functions = [func for name, func in globals().items() if callable(func) and name.startswith("test_")]

# Run the tests
run_tests(test_functions)

## <font color='#669'>1</font>) test_learn_python

### test_learn_python <font color='#699'>Passed</font>

## <font color='#669'>2</font>) test_group_project

### test_group_project <font color='#699'>Passed</font>

## <font color='#669'>3</font>) test_taught_by_ryan

...

### test_absence_6165 <font color='#699'>Passed</font>

## <font color='#669'>10</font>) test_text_book_6156

### <font color='#699'>Debug output</font>

The syllabi you provided list only one textbook:

* **"Machine Learning for Predictive Data Analytics: Algorithms, Worked Examples, and Case Studies"** by John D. Kelleher, Brian Mac Namee, and Aoife D’Arcy, Second Edition, 2020, Massachusetts Institute of Technology. 


### test_text_book_6156 <font color='#699'>Passed</font>